In [1]:
import cv2
import random
import math
import mediapipe as mp
import numpy as np
import os
import re

import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as Data
import torchvision.transforms as transforms
import torch.nn.functional as F
import torch.nn.init as init
from torchvision.models import mobilenet_v3_small
from torch.utils.data import Dataset, DataLoader

from itertools import chain
from sklearn import metrics
import warnings
warnings.filterwarnings("ignore")

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# CFG

In [2]:
time_steps = 3
batch_size = 8
input_dim = 25*4
num_classes = 9
epochs=100

In [3]:
# def seed_torch(seed=45):
#     random.seed(seed)
#     os.environ['PYTHONHASHSEED'] = str(seed)
#     np.random.seed(seed)
#     torch.manual_seed(seed)
#     torch.cuda.manual_seed(seed)
    
# seed_torch()          #42

# 关节坐标提取模型

In [4]:
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

pose = mp_pose.Pose(min_detection_confidence=0.5,
                    min_tracking_confidence=0.5)  # min_detection_confidence表示, min_tracking_confidence表示置信度

# DataSet

### Train_dataset

转化为像素坐标，尝试使用像素坐标作为后面网络的输入。

"""

    for index, lm in enumerate(results.pose_landmarks.landmark):
    h, w, c = img.shape
    
    # 得到的关键点坐标x/y/z/visibility都是比例坐标，在[0,1]之间
    # 转换为像素坐标(cx,cy)，图像的实际长宽乘以比例，像素坐标一定是整数
    cx, cy = int(lm.x * w), int(lm.y * h)

    # 打印坐标信息
    print(index, cx, cy)

"""

In [5]:
folder_path = r'train_dataset_little'
train_video_num =  [1, 2, 3, 4, 6, 7, 8, 9, 10, 11, 12, 13, 14]  # Change to the desired video numbers
train_label_lst=[]

for video_num in train_video_num:
    # Construct the matching pattern for each video number
    pattern_2 = str(video_num) + r'.txt'
    for filename in os.listdir(folder_path):
        # 使用正则表达式匹配文件名
        if re.match(pattern_2, filename):
            # 匹配成功，读取文件
            train_label_path = os.path.join(folder_path, filename)
            train_label_lst.append(train_label_path)
train_label_lst

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


['train_dataset_little/1.txt',
 'train_dataset_little/2.txt',
 'train_dataset_little/3.txt',
 'train_dataset_little/4.txt',
 'train_dataset_little/6.txt',
 'train_dataset_little/7.txt',
 'train_dataset_little/8.txt',
 'train_dataset_little/9.txt',
 'train_dataset_little/10.txt',
 'train_dataset_little/11.txt',
 'train_dataset_little/12.txt',
 'train_dataset_little/13.txt',
 'train_dataset_little/14.txt']

In [6]:
# 获取文件夹中的所有文件名
file_names = os.listdir(folder_path)
img_files = []

# 筛选以 'IMG_' 开头的文件名
for file_name in file_names:
    if file_name.startswith('IMG_'):
        img_files.append(file_name)

# 定义排序函数，按照文件名中的所有数字部分排序
def sort_key(file_name):
    # 使用正则表达式提取文件名中的所有数字部分
    numbers = re.findall(r'\d+', file_name)
    # 将数字部分转换为整数并返回
    return [int(num) for num in numbers]

# 根据文件名中的所有数字部分排序
img_files = sorted(img_files, key=sort_key)

# 逐个读取文件
for img_file in img_files:
    print(img_file)

IMG_1_0000.jpg
IMG_1_0001.jpg
IMG_1_0002.jpg
IMG_1_0003.jpg
IMG_1_0004.jpg
IMG_1_0005.jpg
IMG_1_0006.jpg
IMG_1_0007.jpg
IMG_1_0008.jpg
IMG_1_0009.jpg
IMG_1_0010.jpg
IMG_1_0011.jpg
IMG_1_0012.jpg
IMG_1_0013.jpg
IMG_1_0014.jpg
IMG_1_0015.jpg
IMG_1_0016.jpg
IMG_1_0017.jpg
IMG_1_0018.jpg
IMG_1_0019.jpg
IMG_1_0020.jpg
IMG_1_0021.jpg
IMG_1_0022.jpg
IMG_1_0023.jpg
IMG_1_0024.jpg
IMG_1_0025.jpg
IMG_1_0026.jpg
IMG_1_0027.jpg
IMG_1_0028.jpg
IMG_1_0029.jpg
IMG_1_0030.jpg
IMG_1_0031.jpg
IMG_1_0032.jpg
IMG_1_0033.jpg
IMG_1_0034.jpg
IMG_1_0035.jpg
IMG_1_0036.jpg
IMG_1_0037.jpg
IMG_1_0038.jpg
IMG_1_0039.jpg
IMG_1_0040.jpg
IMG_1_0041.jpg
IMG_1_0042.jpg
IMG_1_0043.jpg
IMG_1_0044.jpg
IMG_1_0045.jpg
IMG_1_0046.jpg
IMG_1_0047.jpg
IMG_1_0048.jpg
IMG_1_0049.jpg
IMG_1_0050.jpg
IMG_1_0051.jpg
IMG_1_0052.jpg
IMG_1_0053.jpg
IMG_1_0054.jpg
IMG_1_0055.jpg
IMG_1_0056.jpg
IMG_1_0057.jpg
IMG_1_0058.jpg
IMG_1_0059.jpg
IMG_1_0060.jpg
IMG_1_0061.jpg
IMG_1_0062.jpg
IMG_1_0063.jpg
IMG_1_0064.jpg
IMG_1_0065.jpg
IMG_1_0066

In [7]:
import numpy as np
from collections import Counter

train_labels = []
for label_path in train_label_lst:
    with open(label_path, 'r', encoding='utf-8') as file:
        content = file.read()
        train_labels.extend(map(int, content.split(',')))  # Convert labels to integers and extend the list

label_counts = Counter(train_labels)  # Use train_labels directly as a one-dimensional list

print(len(train_labels))
print(label_counts)

2952
Counter({7: 771, 2: 313, 1: 300, 3: 299, 0: 286, 4: 279, 5: 249, 8: 239, 6: 216})


In [8]:
train_data_labels = []

# Flatten the train_labels list of lists into a single list

for video_num in train_video_num:
    # Construct the matching pattern for each video number
    pattern = r'IMG_' + str(video_num) + r'_\d+\.jpg'

    # 前14个是txt文件
    for filename, label in zip(img_files, train_labels):
        # Use regular expression to match the filename
        if re.match(pattern, filename):
            # Match successful, construct the file path
            file_path = os.path.join(folder_path, filename)
            # Append the file path and corresponding label to data_labels
            train_data_labels.append((file_path, label))
print(len(train_data_labels))
train_data_labels

2952


[('train_dataset_little/IMG_1_0000.jpg', 7),
 ('train_dataset_little/IMG_1_0001.jpg', 7),
 ('train_dataset_little/IMG_1_0002.jpg', 7),
 ('train_dataset_little/IMG_1_0003.jpg', 7),
 ('train_dataset_little/IMG_1_0004.jpg', 7),
 ('train_dataset_little/IMG_1_0005.jpg', 7),
 ('train_dataset_little/IMG_1_0006.jpg', 3),
 ('train_dataset_little/IMG_1_0007.jpg', 3),
 ('train_dataset_little/IMG_1_0008.jpg', 3),
 ('train_dataset_little/IMG_1_0009.jpg', 3),
 ('train_dataset_little/IMG_1_0010.jpg', 3),
 ('train_dataset_little/IMG_1_0011.jpg', 3),
 ('train_dataset_little/IMG_1_0012.jpg', 3),
 ('train_dataset_little/IMG_1_0013.jpg', 3),
 ('train_dataset_little/IMG_1_0014.jpg', 7),
 ('train_dataset_little/IMG_1_0015.jpg', 7),
 ('train_dataset_little/IMG_1_0016.jpg', 7),
 ('train_dataset_little/IMG_1_0017.jpg', 7),
 ('train_dataset_little/IMG_1_0018.jpg', 7),
 ('train_dataset_little/IMG_1_0019.jpg', 7),
 ('train_dataset_little/IMG_1_0020.jpg', 0),
 ('train_dataset_little/IMG_1_0021.jpg', 0),
 ('train_d

In [9]:
grouped_data = {}
for file_path, label in train_data_labels:
    if label not in grouped_data:
        grouped_data[label] = []
    grouped_data[label].append(file_path)
    
# 对于标签为 7 的数据，下采样到 300
downsampled_data = []
for label, files in grouped_data.items():
    if label == 7:
        # 如果标签为 7，随机选择 300 个文件
        downsampled_files = random.sample(files, 300)
        downsampled_data.extend([(file_path, label) for file_path in downsampled_files])
    else:
        # 其他标签保持不变
        downsampled_data.extend([(file_path, label) for file_path in files])
# 获取下采样后的索引
sampled_indices = [i for i, (file_path, label) in enumerate(train_data_labels) if (file_path, label) in downsampled_data]
# 打印采样后的索引
print(len(sampled_indices))

2481


In [10]:
# 统计下采样后的数据标签分布
label_counts = {}
for _, label in downsampled_data:
    if label not in label_counts:
        label_counts[label] = 0
    label_counts[label] += 1

print("下采样后的数据标签分布：")
for label, count in label_counts.items():
    print(f"标签 {label}: {count} 个样本")

[train_data_labels[x] for x in sampled_indices]

下采样后的数据标签分布：
标签 7: 300 个样本
标签 3: 299 个样本
标签 0: 286 个样本
标签 8: 239 个样本
标签 6: 216 个样本
标签 2: 313 个样本
标签 1: 300 个样本
标签 5: 249 个样本
标签 4: 279 个样本


[('train_dataset_little/IMG_1_0002.jpg', 7),
 ('train_dataset_little/IMG_1_0005.jpg', 7),
 ('train_dataset_little/IMG_1_0006.jpg', 3),
 ('train_dataset_little/IMG_1_0007.jpg', 3),
 ('train_dataset_little/IMG_1_0008.jpg', 3),
 ('train_dataset_little/IMG_1_0009.jpg', 3),
 ('train_dataset_little/IMG_1_0010.jpg', 3),
 ('train_dataset_little/IMG_1_0011.jpg', 3),
 ('train_dataset_little/IMG_1_0012.jpg', 3),
 ('train_dataset_little/IMG_1_0013.jpg', 3),
 ('train_dataset_little/IMG_1_0014.jpg', 7),
 ('train_dataset_little/IMG_1_0015.jpg', 7),
 ('train_dataset_little/IMG_1_0019.jpg', 7),
 ('train_dataset_little/IMG_1_0020.jpg', 0),
 ('train_dataset_little/IMG_1_0021.jpg', 0),
 ('train_dataset_little/IMG_1_0022.jpg', 0),
 ('train_dataset_little/IMG_1_0023.jpg', 0),
 ('train_dataset_little/IMG_1_0024.jpg', 0),
 ('train_dataset_little/IMG_1_0025.jpg', 0),
 ('train_dataset_little/IMG_1_0026.jpg', 0),
 ('train_dataset_little/IMG_1_0027.jpg', 0),
 ('train_dataset_little/IMG_1_0028.jpg', 0),
 ('train_d

### 结合关键点集合，图像，标签

In [11]:
# 定义列表用于存储处理后的数据
train_data = []

# 遍历 test_dataset 中的文件路径
for file_path, label in [train_data_labels[x] for x in sampled_indices]:
    # 使用 OpenCV 加载图像
    img = cv2.imread(file_path)

    # 将图像从 BGR 转换为 RGB
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # 姿势估计处理
    joint = pose.process(img)

    if joint.pose_landmarks is not None:
        joint_lst = []
        for idx, lm in enumerate(joint.pose_landmarks.landmark):
            # 将关节信息添加到列表中
            joint_lst.append([lm.x, lm.y, lm.z, lm.visibility])

        # 将关节坐标转换为 NumPy 数组
        joint_lst = joint_lst[:25]

        # 创建关键点图像
        joint_img = np.zeros((720, 1280), dtype=np.uint8)
        connections = [[0, 1], [1, 2], [2, 3], [4, 5], [5, 6], [3, 7], [6, 8], [9, 10], [11, 12], [11, 13],
                       [13, 15], [12, 14], [14, 16], [15, 17], [15, 21], [17, 19], [15, 19], [16, 22], [16, 20],
                       [18, 20], [16, 18], [12, 24], [11, 23]]
        for connection in connections:
            start_point = (int(joint_lst[connection[0]][0] * 1280), int(joint_lst[connection[0]][1] * 720))
            end_point = (int(joint_lst[connection[1]][0] * 1280), int(joint_lst[connection[1]][1] * 720))
            cv2.line(joint_img, start_point, end_point, 1, thickness=1)
        
        #膨胀    
        # kernel = np.ones((3,3),np.uint8) 
        # joint_img = cv2.dilate(joint_img,kernel,iterations = 10)
        
        # 调整关键点图像大小
        desired_width = 1280  # 调整为所需的宽度
        desired_height = 720  # 调整为所需的高度
        joint_img = cv2.resize(joint_img, (720, 1280))
        joint_img = np.tile(joint_img, (3, 1, 1))
        # 将关键点列表和关键点图像添加到 test_data 中
        train_data.append((joint_lst, joint_img, label))
    else:
        print("No pose landmarks found for file:", file_path)

print("Length of train_data:", len(train_data))

Length of train_data: 2481


### Test_dataset

In [12]:
folder_path = r'test_dataset_little'
test_video_num = 15  # 更改为你想要的视频编号

test_label_lst=[]
pattern_2 = str(test_video_num) + r'.txt'

for filename in os.listdir(folder_path):
    # 使用正则表达式匹配文件名
    if re.match(pattern_2, filename):
        # 匹配成功，读取文件
        test_label_path = os.path.join(folder_path, filename)
        test_label_lst.append(test_label_path)
test_label_lst

['test_dataset_little/15.txt']

In [13]:
test_labels = []
for label_path in test_label_lst:
    with open(label_path, 'r', encoding='utf-8') as file:
        content = file.read()
        test_labels.extend(map(int, content.split(',')))  # Convert labels to integers and extend the list

print(len(test_labels))

210


In [14]:
test_data_labels = []

# 构建匹配文件名的正则表达式
pattern = r'IMG_' + str(test_video_num) + r'_\d+\.jpg'
x = []
# 遍历文件夹中的文件
# 两个txt文件是标签
for filename in sorted(os.listdir(folder_path)[:]):
    # 使用正则表达式匹配文件名
    if re.match(pattern, filename):
        x.append(filename)
    
for filename, label in zip(x, test_labels):
    # 使用正则表达式匹配文件名
    if re.match(pattern, filename):
        # 匹配成功，读取文件
        file_path = os.path.join(folder_path, filename)
        test_data_labels.append((file_path, label))
    else:
        print(filename)
        

# 定义排序函数，按照文件名中的所有数字部分排序
def sort_key(item):
    filename, _ = item
    # 使用正则表达式提取文件名中的所有数字部分
    numbers = re.findall(r'\d+', filename)
    # 将数字部分转换为整数并返回
    return [int(num) for num in numbers]

# 根据文件名中的所有数字部分排序
test_data_labels = sorted(test_data_labels, key=sort_key)

print(len(test_data_labels))
test_data_labels

210


[('test_dataset_little/IMG_15_0000.jpg', 7),
 ('test_dataset_little/IMG_15_0001.jpg', 7),
 ('test_dataset_little/IMG_15_0002.jpg', 7),
 ('test_dataset_little/IMG_15_0003.jpg', 7),
 ('test_dataset_little/IMG_15_0004.jpg', 7),
 ('test_dataset_little/IMG_15_0005.jpg', 7),
 ('test_dataset_little/IMG_15_0006.jpg', 3),
 ('test_dataset_little/IMG_15_0007.jpg', 3),
 ('test_dataset_little/IMG_15_0008.jpg', 3),
 ('test_dataset_little/IMG_15_0009.jpg', 3),
 ('test_dataset_little/IMG_15_0010.jpg', 3),
 ('test_dataset_little/IMG_15_0011.jpg', 3),
 ('test_dataset_little/IMG_15_0012.jpg', 3),
 ('test_dataset_little/IMG_15_0013.jpg', 3),
 ('test_dataset_little/IMG_15_0014.jpg', 3),
 ('test_dataset_little/IMG_15_0015.jpg', 3),
 ('test_dataset_little/IMG_15_0016.jpg', 3),
 ('test_dataset_little/IMG_15_0017.jpg', 3),
 ('test_dataset_little/IMG_15_0018.jpg', 3),
 ('test_dataset_little/IMG_15_0019.jpg', 3),
 ('test_dataset_little/IMG_15_0020.jpg', 7),
 ('test_dataset_little/IMG_15_0021.jpg', 7),
 ('test_da

### 结合关键点集合，图像，标签

In [15]:
# 定义列表用于存储处理后的数据
test_data = []

# 遍历 test_dataset 中的文件路径
for file_path, label in [test_data_labels[x] for x in range(len(test_data_labels))]:
    # 使用 OpenCV 加载图像
    img = cv2.imread(file_path)

    # 将图像从 BGR 转换为 RGB
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # 姿势估计处理
    joint = pose.process(img)

    if joint.pose_landmarks is not None:
        joint_lst = []
        for idx, lm in enumerate(joint.pose_landmarks.landmark):
            # 将关节信息添加到列表中
            joint_lst.append([lm.x, lm.y, lm.z, lm.visibility])

        # 将关节坐标转换为 NumPy 数组
        joint_lst = joint_lst[:25]

        # 创建关键点图像
        joint_img = np.zeros((720, 1280), dtype=np.uint8)
        connections = [[0, 1], [1, 2], [2, 3], [4, 5], [5, 6], [3, 7], [6, 8], [9, 10], [11, 12], [11, 13],
                       [13, 15], [12, 14], [14, 16], [15, 17], [15, 21], [17, 19], [15, 19], [16, 22], [16, 20],
                       [18, 20], [16, 18], [12, 24], [11, 23]]
        for connection in connections:
            start_point = (int(joint_lst[connection[0]][0] * 1280), int(joint_lst[connection[0]][1] * 720))
            end_point = (int(joint_lst[connection[1]][0] * 1280), int(joint_lst[connection[1]][1] * 720))
            cv2.line(joint_img, start_point, end_point, 1, thickness=1)
            
        #膨胀    
        # kernel = np.ones((3,3),np.uint8) 
        # joint_img = cv2.dilate(joint_img,kernel,iterations = 10)

        # 调整关键点图像大小
        joint_img = cv2.resize(joint_img, (720, 1280))
        joint_img = np.tile(joint_img, (3, 1, 1))  # 将图像复制为 [3, 720, 1280]

        # 将关键点列表和关键点图像添加到 test_data 中
        test_data.append((joint_lst, joint_img, label))
    else:
        print("No pose landmarks found for file:", file_path)

print("Length of test_data:", len(test_data))

Length of test_data: 210


In [16]:
# plt.imshow(joint_img[0], cmap='gray')
# plt.axis('off')
# plt.show()

# DataLoader

In [17]:
import torch
import torch.nn as nn

class LabelSmoothingLoss(nn.Module):
    def __init__(self, num_classes, smoothing=0.0):
        super(LabelSmoothingLoss, self).__init__()
        self.num_classes = num_classes
        self.smoothing = smoothing
        self.confidence = 1.0 - smoothing

    def forward(self, pred, target):
        target_one_hot = torch.zeros_like(pred)
        target_one_hot.fill_(self.smoothing / (self.num_classes - 1))
        target_one_hot.scatter_(1, target.unsqueeze(-1).long(), self.confidence)
        
        loss = -torch.sum(target_one_hot * torch.log_softmax(pred, dim=1), dim=1)
        return loss.mean()

In [18]:
class CustomDataset(Dataset):
    def __init__(self, data, time_steps):
        self.data = data
        self.time_steps = time_steps

    def __len__(self):
        return len(self.data) - self.time_steps + 1

    def __getitem__(self, index):
        keypoints_sequence = []
        labels = []
        label_lst = []
        for i in range(self.time_steps):
            keypoints = self.data[index + i][0]
            keypoints_flat = torch.flatten(torch.tensor(keypoints)).unsqueeze(1)
            keypoints_sequence.append(keypoints_flat)
            labels.append(self.data[index + i][2])
        keypoints_sequence = torch.cat(keypoints_sequence, dim=1)
        joint_img = self.data[index + self.time_steps - 1][1]

        return keypoints_sequence, joint_img, labels


In [19]:
train_dataset = CustomDataset(train_data, time_steps)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True,drop_last=True)
# 遍历数据集的每个批次
test_dataset = CustomDataset(test_data, time_steps)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [20]:
for batch_idx, (keypoints_sequence,joint_img, labels) in enumerate(train_dataloader):
    # Access the first batch
    first_batch = (keypoints_sequence,joint_img, labels)
    break

keypoints_sequence,joint_img, labels = first_batch
print(keypoints_sequence.shape)
print(labels)
joint_img.shape

torch.Size([8, 100, 3])
[tensor([0, 1, 0, 5, 1, 4, 6, 2]), tensor([0, 1, 0, 5, 1, 4, 6, 2]), tensor([0, 1, 0, 5, 1, 4, 6, 2])]


torch.Size([8, 3, 1280, 720])

# Model

In [21]:
# def label_smooth(labels, smoothing=0.2):    
#     path = torch.zeros(8)
    
#     for i in range(0, 8):
#         if labels[0][i] == labels[1][i] == labels[2][i]:
#             path[i] = 3
#         elif labels[0][i] == labels[1][i]:
#             path[i] = 2
#         elif labels[1][i] == labels[2][i]:
#             path[i] = 1
    
#     label_matrix_1 = torch.zeros((len(labels[0]), num_classes))
#     label_matrix_2 = torch.zeros((len(labels[0]), num_classes))
    
#     for i in range(len(labels[0])):
#         if path[i] == 0 or path[i] == 3:
#             label_matrix_1[i][labels[2][i]] = 1
#         elif path[i] == 2:
#             label_matrix_1[i][labels[2][i]] = 0.6
#             label_matrix_2[i][labels[1][i]] = 0.4
#         elif path[i] == 1:
#             label_matrix_1[i][labels[2][i]] = 0.7
#             label_matrix_2[i][labels[0][i]] = 0.3
    
#     label_matrix = torch.add(label_matrix_1, label_matrix_2)
    
#     return path, label_matrix

# path,label_matrix=label_smooth(labels)
# print(labels)
# print("path:",path)
# print("labels:",label_matrix)

In [22]:
def label_smooth(labels):    
    path = torch.zeros(8)
    
    for i in range(0, 8):
        if labels[0][i] == labels[1][i] == labels[2][i]:
            path[i] = 3
        elif labels[0][i] == labels[1][i]:
            path[i] = 2
        elif labels[1][i] == labels[2][i]:
            path[i] = 1
    
    label_matrix_1 = torch.zeros((len(labels[0]), num_classes))
    label_matrix_2 = torch.zeros((len(labels[0]), num_classes))
    
    for i in range(len(labels[0])):
        if path[i] == 0 or path[i] == 3:
            label_matrix_1[i][labels[2][i]] = 1
        elif path[i] == 2:
            label_matrix_1[i][labels[2][i]] = 0.9
            label_matrix_2[i][labels[1][i]] = 0.1
        elif path[i] == 1:
            label_matrix_1[i][labels[2][i]] = 0.95
            label_matrix_2[i][labels[0][i]] = 0.05
            
            
    
    label_matrix = torch.add(label_matrix_1, label_matrix_2)
    
    return label_matrix

In [23]:
class Mobile_LSTMModel(nn.Module):
    def __init__(self, num_classes):
        super(Mobile_LSTMModel, self).__init__()
        # Define CNN layers
        self.conv1 = nn.Conv1d(in_channels=100, out_channels=100, kernel_size=3, padding=1)
        self.conv2 = nn.Conv1d(in_channels=100, out_channels=128, kernel_size=3, padding=1)
        self.conv3 = nn.Conv1d(in_channels=128, out_channels=256, kernel_size=3, padding=1)
        
        # Define BatchNorm layers
        self.bn1 = nn.BatchNorm1d(100)
        self.bn2 = nn.BatchNorm1d(128)
        self.bn3 = nn.BatchNorm1d(256)
        
        # Use MSRA initialization for convolutional layers
        for conv_layer in [self.conv1, self.conv2, self.conv3]:
            init.kaiming_normal_(conv_layer.weight, mode='fan_out', nonlinearity='relu')
        
        # Define LSTM layer
        self.lstm = nn.LSTM(input_size=256, hidden_size=512, num_layers=2, dropout=0.1, batch_first=True)
        
        # SVD orthogonalization for LSTM parameters
        for name, param in self.lstm.named_parameters():
            if 'weight' in name:
                weight = param.data  # Get weight parameters
                _, s, v = torch.svd(weight)  # Perform SVD decomposition
                param.data = torch.mm(torch.mm(_, torch.diag(s)), v.t())  # Reconstruct orthogonalized weight
        
        # Initialize hidden state
        hidden_size = self.lstm.hidden_size
        num_layers = self.lstm.num_layers
        weight = next(self.lstm.parameters()).data
        h_0 = weight.new(num_layers, batch_size, hidden_size).normal_(0, 1)
        c_0 = weight.new(num_layers, batch_size, hidden_size).normal_(0, 1)
        self.hidden = (h_0, c_0)
        
        # Define MobileNet V3 network
        self.mobilenet = mobilenet_v3_small(pretrained=False)
        
        self.conv4 = nn.Conv1d(in_channels=1000, out_channels=256, kernel_size=3, padding=1)
        self.conv5 = nn.Conv1d(in_channels=256+512, out_channels=512, kernel_size=3, padding=1)
        self.conv6 = nn.Conv1d(in_channels=512, out_channels=512, kernel_size=3, padding=1)
        
        # Define BatchNorm layers
        self.bn4 = nn.BatchNorm1d(256)
        self.bn5 = nn.BatchNorm1d(512)
        self.bn6 = nn.BatchNorm1d(512)
        
        # Define FC layer
        self.fc = nn.Linear(512, num_classes)  # 512 from LSTM + 1000 from MobileNet V3
        
    def forward(self, x, mobilenet_input):
        # CNNLSTMModel forward pass
        x = self.conv1(x)
        x = self.bn1(x)
        x = F.relu(x)

        x = self.conv2(x)
        x = self.bn2(x)
        x = F.relu(x)

        x = self.conv3(x)
        x = self.bn3(x)
        x = F.relu(x)

        x = x.permute(0, 2, 1)  # Reshape dimensions to [batch_size, 256, new_seq_len]
        lstm_output, _ = self.lstm(x)
        lstm_output = lstm_output[:, -1, :]  # Take the last time step's output

        # MobileNet V3 forward pass
        mobilenet_output = self.mobilenet(mobilenet_input)

        # Concatenate the outputs of CNN and MobileNet
        mobilenet_output = mobilenet_output.unsqueeze(2)
        mobilenet_output = self.conv4(mobilenet_output)
        mobilenet_output = self.bn4(mobilenet_output)
        mobilenet_output = F.relu(mobilenet_output)
        mobilenet_output = mobilenet_output.squeeze(2)
        
        combined_output = torch.cat((lstm_output, mobilenet_output), dim=1)

        # Additional convolutional layers
        x = combined_output.unsqueeze(2)  # Add a singleton dimension for conv1d
        x = self.conv5(x)
        x = self.bn5(x)
        x = F.relu(x)

        x = self.conv6(x)
        x = self.bn6(x)
        x = F.relu(x)

        # Fully connected layer forward pass with ReLU activation
        x = x.squeeze(2)  # Remove the singleton dimension before FC layer
        output = self.fc(x)

        return output

# Create CNN+LSTM model instance
num_classes = 9

# Train

In [44]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = Mobile_LSTMModel(num_classes)
model = model.to(device)
t =  10 # warmup
T = 150  # 共有120个epoch，则用于cosine rate的一共有110个epoch
lr_rate = 6e-4
n_t = 1# 调整这个值以减缓学习率的衰减速率
best=1
lambda1 = lambda epoch: (0.9 * epoch / t + 0.1) if epoch < t else 0.1 if n_t * (1 + math.cos(math.pi * (epoch - t) / (T - t))) < 0.1 else n_t * (1 + math.cos(math.pi * (epoch - t) / (T - t)))
optimizer = optim.AdamW(model.parameters(), lr=lr_rate, weight_decay=1e-2)
scheduler = optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=lambda1)
criterion = nn.CrossEntropyLoss()

best_test_accuracy = 0.0
best_epoch = 0
best_train_accuracy = 0.0
predict_label_lst=[]

# 训练模型
for epoch in range(epochs):
    total_loss = 0.0
    total_correct = 0
    model.train()

    for batch_idx, (points,img, labels) in enumerate(train_dataloader):
        points = points.to(device)
        img = img.to(device).float()
        label_tensor = label_smooth(labels).to(device)
        label = labels[2].to(device)
        # print(label_tensor)
        
        # Create a matrix of zeros


        optimizer.zero_grad()
        outputs = model(points,img)
        # print(outputs)
        
        loss = criterion(outputs, label_tensor)
        loss.backward()
        optimizer.step()
        scheduler.step() 

        # 统计总的loss和正确预测的样本数
        total_loss += loss.item() * points.size(0)
        _, predicted = torch.max(outputs, 1)
        total_correct += (predicted == label).float().sum().item()
        # 每迭代100次打印lr、loss、acc等参数值
        if (batch_idx + 1) % 100 == 0:
            current_lr = optimizer.param_groups[0]['lr']
            epoch_loss = total_loss / ((batch_idx + 1) * points.size(0))
            epoch_accuracy = total_correct / ((batch_idx + 1) * points.size(0))
            print(f"Epoch {epoch+1}/{epochs}, Iteration {batch_idx+1}, LR: {current_lr}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.4f}")

    # 输出每个epoch的平均loss和准确率
    epoch_loss = total_loss / len(train_dataloader.dataset)
    epoch_accuracy = total_correct / len(train_dataloader.dataset)
    print("*******************************************************")
    print(f"Epoch {epoch+1}/{epochs}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.4f}")

    # 设置模型为评估模式
    model.eval()
    total_test_correct = 0
    test_predictions=[]
    test_labels=[]
    with torch.no_grad():  # 关闭梯度计算
        for test_batch_idx, (test_data,test_img, test_label) in enumerate(test_dataloader):
            test_points = test_data.to(device)
            test_img = test_img.to(device).float()
            test_label = test_label[2].to(device)  # 将测试集标签数组转换为张量

            # 前向传播
            test_outputs = model(test_points,test_img)
            _, test_predicted = torch.max(test_outputs, 1)
            total_test_correct += (test_predicted == test_label).sum().item()
            test_predictions.extend(test_predicted.tolist())
            test_labels.extend(test_label.tolist())

        test_accuracy = total_test_correct / len(test_dataloader.dataset)
        print(f"Epoch {epoch+1}/{epochs}, Test Accuracy: {test_accuracy:.4f}")
        print("*******************************************************")

            # Compute evaluation metrics
        test_predictions = np.array(test_predictions)
        test_labels = np.array(test_labels)
        precision = metrics.precision_score(test_labels, test_predictions, average='weighted', zero_division=0)
        recall = metrics.recall_score(test_labels, test_predictions, average='weighted', zero_division=0)
        f1_score = metrics.f1_score(test_labels, test_predictions, average='weighted', zero_division=0)
#         auc = metrics.roc_auc_score(test_labels, test_predictions, multi_class='ovr')

        # Print evaluation metrics
        print(f"Epoch {epoch+1}/{epochs}, Test Precision: {precision:.4f}")
        print(f"Epoch {epoch+1}/{epochs}, Test Recall: {recall:.4f}")
        print(f"Epoch {epoch+1}/{epochs}, Test F1-score: {f1_score:.4f}")
#         print(f"Epoch {epoch+1}/{epochs}, Test AUC: {auc:.4f}")

    
        test_predictions = np.array(test_predictions)
        test_labels = np.array(test_labels)

        # 计算每个类别的准确率
        class_accuracy = metrics.precision_score(test_labels, test_predictions, average=None)

        # 计算除标签7以外的其他标签的平均准确率
        average_accuracy = metrics.precision_score(test_labels, test_predictions, average='weighted', labels=np.unique(test_labels[test_labels != 7]))

        # 打印每个类别的准确率
        for class_label, accuracy in enumerate(class_accuracy):
            print(f"Label {class_label}: {accuracy:.4f}")

        # 打印除了标签7以外的其他标签的平均准确率
        print(f"Average Accuracy (excluding label 7): {average_accuracy:.4f}")
        print("*******************************************************")
        
        # Check if current test accuracy is the best so far
        if test_accuracy > best_test_accuracy:
            best_test_accuracy = test_accuracy
            best_epoch = epoch + 1
            best_train_accuracy = epoch_accuracy
            predict_label_lst=test_predictions
            if test_accuracy > 0.87:
                s="best_model_"+str(best)+".pth"
                # Define the path where you want to save the model
                model_path = s
                best+=1
                # Save the model's state dictionary
                torch.save(model.state_dict(), model_path)

Epoch 1/100, Iteration 100, LR: 0.0003396697565294651, Loss: 1.8013, Accuracy: 0.3438
Epoch 1/100, Iteration 200, LR: 0.000339669756529465, Loss: 1.6353, Accuracy: 0.3975
Epoch 1/100, Iteration 300, LR: 0.001184956747309094, Loss: 1.6390, Accuracy: 0.3921
*******************************************************
Epoch 1/100, Loss: 1.6377, Accuracy: 0.3893
Epoch 1/100, Test Accuracy: 0.2837
*******************************************************
Epoch 1/100, Test Precision: 0.1680
Epoch 1/100, Test Recall: 0.2837
Epoch 1/100, Test F1-score: 0.1885
Label 0: 0.0000
Label 1: 1.0000
Label 2: 0.0000
Label 3: 0.0000
Label 4: 0.0000
Label 5: 0.0000
Label 6: 0.0000
Label 7: 0.2966
Label 8: 0.0000
Average Accuracy (excluding label 7): 0.1304
*******************************************************
Epoch 2/100, Iteration 100, LR: 6.539608548697959e-05, Loss: 1.4435, Accuracy: 0.4612
Epoch 2/100, Iteration 200, LR: 0.0007203536494699639, Loss: 1.4039, Accuracy: 0.4706
Epoch 2/100, Iteration 300, LR: 

In [45]:
print(f"Best Test Accuracy: {best_test_accuracy:.4f} at Epoch {best_epoch}, Train Accuracy: {best_train_accuracy:.4f}")
predict_label_lst

Best Test Accuracy: 0.8894 at Epoch 35, Train Accuracy: 0.8713


array([7, 7, 7, 7, 7, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 7, 7, 7, 7,
       7, 7, 4, 7, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 7,
       7, 7, 7, 7, 7, 7, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       7, 7, 7, 0, 3, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 3, 3, 3, 0, 0, 0,
       0, 4, 7, 7, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6,
       6, 6, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 8, 8, 8, 8, 8, 8, 8, 8, 8,
       8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 7, 7, 7,
       7, 7, 7, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 7, 7, 7, 7, 7, 7, 7, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

Epoch 35/100, Loss: 0.4100, Accuracy: 0.8713

Epoch 35/100, Test Accuracy: 0.8894

*******************************************************

Epoch 35/100, Test Precision: 0.8912

Epoch 35/100, Test Recall: 0.8894

Epoch 35/100, Test F1-score: 0.8886

Label 0: 0.7333

Label 1: 0.9500

Label 2: 0.9583

Label 3: 0.7222

Label 4: 0.9474

Label 5: 0.9375

Label 6: 0.9615

Label 7: 0.8333

Label 8: 0.9286

Average Accuracy (excluding label 7): 0.9081

[7, 7, 7, 7, 7, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 7, 7, 7, 7,
       7, 7, 4, 7, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 7,
       7, 7, 7, 7, 7, 7, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       7, 7, 7, 0, 3, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 3, 3, 3, 0, 0, 0,
       0, 4, 7, 7, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6,
       6, 6, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 8, 8, 8, 8, 8, 8, 8, 8, 8,
       8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 7, 7, 7,
       7, 7, 7, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 7, 7, 7, 7, 7, 7, 7, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2]